In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import cv2
import mediapipe as mp
import time
from scipy import signal
import scipy
import jade
import torchvision
import torch.nn.functional as F

In [3]:
def filtering(data,fs,lowerCutoff=1, higherCutoff=3, filterOrder=5):
    lowerCutoffDigital = lowerCutoff / (0.5 * fs)
    higherCutoffDigital = higherCutoff / (0.5 * fs)
    b, a = signal.butter(filterOrder, [lowerCutoffDigital, higherCutoffDigital], btype='band', analog=False)
    filtsignal = signal.filtfilt(b, a, data)
    return filtsignal
def bandpass_filter(data,fps, lowcut, highcut):
    fs = fps # Частота дискретизации (количество измерений сигнала в 1 сек)
    nyq = 0.5 * fs # Частота Найквиста
    low = float(lowcut) / float(nyq)
    high = float(highcut) / float(nyq)
    order = 6.0 # Номер фильтра в scipy.signal.butter
    b, a = butter(order, [low, high], btype='band')
    bandpass = lfilter(b, a, data)
    return bandpass
def Signal_average(PMatrix):
    signal=np.average(PMatrix,axis=0)
    return signal

def HR_FINDER(wave,fs):
    fftData = np.fft.fft(wave,1000)[0:500]
    hz=np.linspace(0,fs/2,int(len(fftData)))
    powerSpectrum = np.abs(fftData)**2
    maxFreq = np.argmax(powerSpectrum)
    HR=hz[maxFreq]*60
    return HR


# SKIN_SEGMENTATION

In [4]:
import numpy as np
import torch.nn as nn
import torch
import pytorch_lightning as pl
import cv2
import matplotlib.pyplot as plt

In [5]:
class decoder_seg(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(512,512,kernel_size=3,padding=1)
        self.conv2=nn.Conv2d(512,512,kernel_size=3,padding=1)
        self.conv3=nn.Conv2d(512,512,kernel_size=3,padding=1)
        self.conv4=nn.Conv2d(512,256,kernel_size=3,padding=1)
        self.conv5=nn.Conv2d(256,256,kernel_size=3,padding=1)
        self.conv6=nn.Conv2d(256,128,kernel_size=3,padding=1)
        self.conv7=nn.Conv2d(128,128,kernel_size=3,padding=1)
        self.conv8=nn.Conv2d(128,64,kernel_size=3,padding=1)
        self.conv9=nn.Conv2d(64,64,kernel_size=3,padding=1)
        self.conv10=nn.Conv2d(64,4,kernel_size=3,padding=1)
        self.relu=nn.ReLU()

    def forward(self,x):
        x=F.interpolate(x, scale_factor=2)
        x=self.relu(self.conv1(x))
        x=self.relu(self.conv2(x))
        x=F.interpolate(x,scale_factor=2)
        x=self.relu(self.conv3(x))
        x=self.relu(self.conv4(x))
        x=F.interpolate(x,scale_factor=2)
        x=self.relu(self.conv5(x))
        x=self.relu(self.conv6(x))
        x=F.interpolate(x,scale_factor=2)
        x=self.relu(self.conv7(x))
        x=self.relu(self.conv8(x))
        x=F.interpolate(x,scale_factor=2)
        x=self.relu(self.conv9(x))
        x=self.relu(self.conv10(x))
        
        return x
    

In [6]:
class Baby_skin(pl.LightningModule):
    def __init__(self):
        super().__init__()
        model=torchvision.models.vgg13(weights=torchvision.models.VGG13_Weights)
        self.encoder_seg=nn.Sequential(*list(model.children())[0])
        self.decoder_seg=decoder_seg()
        self.loss_fun=nn.CrossEntropyLoss()
        self.optimizer=torch.optim.Adam(self.decoder_seg.parameters(),lr=1e-4)
        
    def forward(self,x):
        x=self.encoder_seg(x)
        x=self.decoder_seg(x)
        return x
    def training_step(self,batch,batch_index):
        img,mask=batch
        pred=self(img)
        mask=mask[:,0,:,:]
        
        loss=self.loss_fun(pred,mask)
        pred=torch.argmax(pred,dim=1)
        
        train_acc=(pred==mask).sum()/(mask.numel())
        
        self.log('Training_loss',loss)
        self.log('Training_acc',train_acc)
        return loss
        
    def validation_step(self,batch,batch_indx):
        img,mask=batch
        pred=self(img)
        mask=mask[:,0,:,:]
        
        loss=self.loss_fun(pred,mask)
        
        pred=torch.argmax(pred,dim=1)
        
        val_acc=(pred==mask).sum()/(mask.numel())
        
        self.log('Validation_loss',loss)
        self.log('Val_acc',val_acc)
        return loss
    
    def configure_optimizers(self):
        return [self.optimizer]

In [7]:
def cuda_img(img):
    img=cv2.resize(img,(224,224))
    img=np.transpose(img,(2,0,1))
    img=img/255
    img=torch.tensor(img)
    img=img.float()
    img=img.unsqueeze(0)
    img=img.to('cuda')
    return img
def view_img(img,mask):
    for i in range(1,4):
        new_mask=np.zeros_like(mask)
        new_mask[mask==i]=255
        new_mask=np.uint8(new_mask)
        new_mask=cv2.resize(new_mask,(640,480))
        contour,_=cv2.findContours(new_mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        img=cv2.drawContours(img,contour,-1,color[i],2)
    return img

def new_mask(mask,indx):
    new_mask=np.zeros_like(mask)
    new_mask[mask==indx]=255
    new_mask=np.uint8(new_mask)
    new_mask=cv2.resize(new_mask,(640,480))
    return new_mask

In [8]:
model=Baby_skin.load_from_checkpoint(r'C:\Users\PhysioSens\Desktop\python-jupyter\ubfc_seg\mix\logs\lightning_logs\version_6\checkpoints\epoch=29-step=5940.ckpt')
model=model.to('cuda')
model=model.eval()

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG13_Weights.IMAGENET1K_V1`. You can also use `weights=VGG13_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
dic_label={'forehead':1,'left_cheek':2,'right_cheek':3}
color={1:(0,0,255),2:(255,0,0),3:(0,255,0),4:(255,255,0),5:(255,0,255),6:(0,255,255)}

In [10]:
def POS(matrix):
    pos_matrix=[]
    for z in range(matrix.shape[0]):
        if np.average(matrix[z])==0:
            print('front_hair')
            continue
        color=np.zeros((3,150))
        red=matrix[z,:,2]
        green=matrix[z,:,1]
        blue=matrix[z,:,0]
        color[0,:]=red
        color[1,:]=green
        color[2,:]=blue
        p=np.array([[0,1,-1],[-2,1,1]])
        N=150
        H=np.zeros((N))
        l=42
        for i in range(0,N):
            m=i-l+1
            if m>0:
                data=color[:,m:i].copy()
                #.copy()
                data[0,:]=data[0,:]/(np.mean(data[0,:]))
                data[1,:]=data[1,:]/(np.mean(data[1,:]))
                data[2,:]=data[2,:]/(np.mean(data[2,:]))
                S=p@data
                g1=S.copy()
                alpha=np.std(S[0,:])/np.std(S[1,:])
                s1=S[0,:]
                s2=S[1,:]
                s1=filtering(s1,fs,.7,3)
                s2=filtering(s2,fs,.7,3)
                h=s1+(alpha*s2)
                H[m:i]=H[m:i]+(h-np.mean(h))/np.std(h)
        pos_matrix.append(H)
    return np.array(pos_matrix)
def CHROM(matrix):
    chrom_matrix=[]
    for z in range(matrix.shape[0]):
        if np.average(matrix[z])==0:
            print('front_hair')
            continue
        color=np.zeros((3,150))
        red=matrix[z,:,2]/np.mean(matrix[z,:,2])
        green=matrix[z,:,1]/np.mean(matrix[z,:,1])
        blue=matrix[z,:,0]/np.mean(matrix[z,:,0])
        color[0,:]=red
        color[1,:]=green
        color[2,:]=blue
        chrom=np.array([[3,-2,0],[1.5,1,-1.5]])
        S=chrom@color
        s1=S[0,:]
        s2=S[1,:]
        s1=filtering(s1,fs,.7,3)
        s2=filtering(s2,fs,.7,3)
        alpha=np.std(s1)/np.std(s2)
        pulse=s1-(alpha*s2)
        chrom_matrix.append(pulse)
    return np.array(chrom_matrix)

def GREEN(matrix):
    green_matrix=[]
    for z in range(matrix.shape[0]):
        green=matrix[z,:,1]
        greend=scipy.signal.detrend(green)
        greenN=greend/np.std(greend)
        greenf=filtering(greenN,fs,.7,3)
        green_matrix.append(greenf)
    return np.array(green_matrix)

def ICA(matrix):
    ica_matrix=[]
    for z in range(matrix.shape[0]):
        if np.average(matrix[z])==0 or np.any(np.isnan(matrix[z])):
            print('front_hair')
            continue
        tic=time.time()
        red=matrix[z,:,2]
        green=matrix[z,:,1]
        blue=matrix[z,:,0]
        region1d=scipy.signal.detrend(red)
        region2d=scipy.signal.detrend(blue)
        region3d=scipy.signal.detrend(green)
        region1N=(region1d-(np.mean(region1d)))/(np.std(region1d))
        region2N=(region2d-(np.mean(region2d)))/(np.std(region2d))
        region3N=(region3d-(np.mean(region3d)))/(np.std(region3d))
        region1f=filtering(region1N,fs,lowerCutoff=.7, higherCutoff=3, filterOrder=5)
        region2f=filtering(region2N,fs,lowerCutoff=.7, higherCutoff=3, filterOrder=5)
        region3f=filtering(region3N,fs,lowerCutoff=.7, higherCutoff=3, filterOrder=5)
        filtdata=np.zeros((3,len(region1f)))
        filtdata[0,:]=region1f
        filtdata[1,:]=region2f
        filtdata[2,:]=region3f
        b = jade.main(filtdata)[0]
        iscores = np.array(b)@filtdata
        nyquist = int(len(iscores[0,:])/2)
        outFr=fs
        x_disp = outFr/2*np.arange(nyquist)/nyquist
        # Selecte Best Component. Usually the Second one i.e Index = 1
        powerRatio = []
        listForDistanceEstimation = []
        for i in range(3):
            fftData = np.fft.fft(iscores[i,:])[0:nyquist]
            powerSpectrum = np.abs(fftData)**2
            maxFreq = np.argmax(powerSpectrum)
            powerInMaxFreq = np.sum(powerSpectrum[maxFreq-1:maxFreq+2]) + np.sum(powerSpectrum[2*maxFreq:2*maxFreq+3])
            powerRatio.append(powerInMaxFreq/np.sum(powerSpectrum))
            listForDistanceEstimation.append((maxFreq)/nyquist*outFr/2)
        # Choose Signal 
        PCAIndex = np.argmax(np.array(powerRatio))
        chosenSignal = iscores[PCAIndex,:]
        ica_matrix.append(chosenSignal)
        toc=time.time()
    return np.array(ica_matrix)

# with mediapipe

In [11]:
mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=1,color=(0,128,0))
DATA_path=Path(r'D:\DL_DATA\ddp_cphys\UBFC\DATASET_2')
num1=[10]
num2=[50,280]
ground=[]
pos_hr_full,chrom_hr_full,ica_hr_full=[],[],[]
for sub in os.listdir(DATA_path):
    full_path=DATA_path/sub
    ext=os.listdir(full_path)
    label_path=full_path/ext[0]
    vid_path=full_path/ext[1]
    vid=cv2.VideoCapture(str(vid_path))
    label=np.loadtxt(label_path)
    fs=vid.get(cv2.CAP_PROP_FPS)
    matrix=np.zeros((len(num1)+len(num2),150,3))
    count=0
    flag=0
    for i in range(int(vid.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret,frame=vid.read()
        if ret==False:
            break
            
        if i%150==0 and i!=0:
            wave=label[0][i-150:i]
            wave=scipy.signal.detrend(wave)
            hr=label[1][i-150:i]
            sub_label=(wave,hr)
            pos_matrix=POS(matrix)
            chrom_matrix=CHROM(matrix)
            ica_matrix=ICA(matrix)
            matrix=np.zeros((len(num1)+len(num2),150,3))
            BEST_POS=Signal_average(pos_matrix)
            BEST_CHROM=Signal_average(chrom_matrix)
            #BEST_ICA=Signal_average(ica_matrix)
    
            pos_hr=HR_FINDER(BEST_POS,30)
            chrom_hr=HR_FINDER(BEST_CHROM,30)
            ground.append(HR_FINDER(wave,30))
            #ica_hr_full.append(HR_FINDER(BEST_ICA,30))
            
            pos_hr_full.append(pos_hr)
            chrom_hr_full.append(chrom_hr)
            flag,count=0,0
        img=cuda_img(frame)
        
        with torch.no_grad():
            pred=model(img)
        pred=pred[0].cpu().detach().numpy()
        pred=np.argmax(pred,axis=0)
        id=np.unique(pred)
        
        
        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = faceMesh.process(imgRGB)
        face=[]
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                for id,lm in enumerate(faceLms.landmark):
                    ih, iw, ic = frame.shape
                    x,y = int(lm.x*iw), int(lm.y*ih)
                    face.append((x,y))
                    if id in num1:
                        forehead_size=frame[face[id][1]-25:face[id][1]+25,face[id][0]-60:face[id][0]+60,0].size
                    if id ==280:
                        left_size=frame[face[id][1]-25:face[id][1]+25,face[id][0]-25:face[id][0]+25,0].size
                    if id==50:
                        right_size=frame[face[id][1]-25:face[id][1]+25,face[id][0]-25:face[id][0]+25,0].size
        
        
        forehead_mask=new_mask(pred,1)
        if (frame[forehead_mask==255]).size/forehead_size >0.3:
            
            matrix[flag,count,2]=np.mean(frame[forehead_mask==255][:,2])
            matrix[flag,count,1]=np.mean(frame[forehead_mask==255][:,1])
            matrix[flag,count,0]=np.mean(frame[forehead_mask==255][:,0])
        else:
            print('forehead',sub)
        flag+=1
        
        left_mask=new_mask(pred,2)
        if (frame[left_mask==255]).size/left_size>0.3:
            matrix[flag,count,2]=np.mean(frame[left_mask==255][:,2])
            matrix[flag,count,1]=np.mean(frame[left_mask==255][:,1])
            matrix[flag,count,0]=np.mean(frame[left_mask==255][:,0])
        else:
            print('left',sub)
        flag+=1
        
        right_mask=new_mask(pred,3)
        if (frame[right_mask==255]).size/right_size>0.3:
            matrix[flag,count,2]=np.mean(frame[right_mask==255][:,2])
            matrix[flag,count,1]=np.mean(frame[right_mask==255][:,1])
            matrix[flag,count,0]=np.mean(frame[right_mask==255][:,0])
        else:
            print('right',sub)
        flag+=1
        
        count+=1
        flag=0
        
        img_view=view_img(frame.copy(),pred)
        cv2.imshow('frame',img_view)
        key=cv2.waitKey(1)&0XFF
        if key==ord('q'):
            break
cv2.destroyAllWindows()
        

C:\Users\PhysioSens\AppData\Roaming\Python\Python39\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject11
forehead subject37
left subject37
right subject37
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38


forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
front_hair
front_hair
front_hair
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject3

forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead sub

forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead sub

forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead subject38
forehead sub

In [12]:
predict_hr_fft_all = np.array(chrom_hr_full)
gt_hr_fft_all = np.array(ground)
num_test_samples = len(predict_hr_fft_all)
METRIC=[]
for metric in ['MAE','RMSE','MAPE','Pearson', 'SNR']:
    if metric == "MAE":
        MAE_FFT = np.mean(np.abs(predict_hr_fft_all - gt_hr_fft_all))
        standard_error = np.std(np.abs(predict_hr_fft_all - gt_hr_fft_all)) / np.sqrt(num_test_samples)
        METRIC.append(MAE_FFT)
        print("FFT MAE (FFT Label): {0} +/- {1}".format(MAE_FFT, standard_error))
    elif metric == "RMSE":
        RMSE_FFT = np.sqrt(np.mean(np.square(predict_hr_fft_all - gt_hr_fft_all)))
        standard_error = np.std(np.square(predict_hr_fft_all - gt_hr_fft_all)) / np.sqrt(num_test_samples)
        print("FFT RMSE (FFT Label): {0} +/- {1}".format(RMSE_FFT, standard_error))
        METRIC.append(RMSE_FFT)
    elif metric == "MAPE":
        MAPE_FFT = np.mean(np.abs((predict_hr_fft_all - gt_hr_fft_all) / gt_hr_fft_all)) * 100
        standard_error = np.std(np.abs((predict_hr_fft_all - gt_hr_fft_all) / gt_hr_fft_all)) / np.sqrt(num_test_samples) * 100
        print("FFT MAPE (FFT Label): {0} +/- {1}".format(MAPE_FFT, standard_error))
        METRIC.append(MAPE_FFT)
    elif metric == "Pearson":
        Pearson_FFT = np.corrcoef(predict_hr_fft_all, gt_hr_fft_all)
        correlation_coefficient = Pearson_FFT[0][1]
        standard_error = np.sqrt((1 - correlation_coefficient**2) / (num_test_samples - 2))
        METRIC.append(Pearson_FFT)
        print("FFT Pearson (FFT Label): {0} +/- {1}".format(correlation_coefficient, standard_error))

FFT MAE (FFT Label): 2.152467410643337 +/- 0.3667623777468499
FFT RMSE (FFT Label): 8.612617820881294 +/- 29.678935435934207
FFT MAPE (FFT Label): 2.0766588928770524 +/- 0.30455023196334574
FFT Pearson (FFT Label): 0.8849429625940455 +/- 0.020521168467047848
